## 가설검정의 이해

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import joblib
import numpy as np
import pandas as pd

In [ ]:
# 통계 관련 라이브러리를 호출합니다.
from scipy import stats
import pingouin as pg
import scikit_posthocs as sp
from statsmodels.stats.proportion import proportions_ztest

### 작업 경로 확인 및 변경

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

### 실습 데이터셋 준비

In [ ]:
# z 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price.z')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 공분산

In [ ]:
# 두 연속형 변수의 공분산을 반환합니다.
df['Age'].cov(df['Price'])

In [ ]:
# df의 연속형 변수로 분산-공분산 행렬을 반환합니다.
df.cov(numeric_only = True)

### 상관계수

In [ ]:
# 두 연속형 변수의 피어슨 상관계수를 반환합니다.
df['Age'].corr(df['Price'])

In [ ]:
# df의 연속형 변수로 피어슨 상관계수 행렬을 반환합니다.
df.corr(numeric_only = True)

### 피어슨 상관분석

In [ ]:
# Age와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['Age'], y = df['Price'])

In [ ]:
# KM와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['KM'], y = df['Price'])

In [ ]:
# HP와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['HP'], y = df['Price'])

In [ ]:
# CC와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['CC'], y = df['Price'])

In [ ]:
# Doors와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['Doors'], y = df['Price'])

In [ ]:
# Weight와 Price의 피어슨 상관분석을 실행하고 유의확률을 확인합니다.
pg.corr(x = df['Weight'], y = df['Price'])

### [참고] 피어슨 상관분석 유의확률 출력 함수 생성

In [ ]:
# 변수 x에 df의 연속형 변수를 할당합니다.
x = df['Age']

In [ ]:
# 변수 x와 Price의 피어슨 상관분석 실행 결과에서 유의확률만 출력합니다.
pg.corr(x = x, y = df['Price'])['p-val']

In [ ]:
# 연속형 입력변수와의 상관분석 유의확률을 출력하는 람다 표현식을 생성합니다.
corr = lambda x: pg.corr(x = x, y = df['Price'])['p-val']

In [ ]:
# 람다 표현식 함수로 피어슨 상관분석 유의확률을 출력합니다.
corr(x = df['Age'])

### [참고] apply() 함수를 활용한 상관분석 실행

In [ ]:
# df의 열별 자료형을 확인합니다.
df.dtypes

In [ ]:
# 열별 자료형이 정수형 또는 실수형이면 True, 아니면 False인 벡터를 생성합니다.
locs = df.dtypes.astype(str).isin(values = ['float64', 'int64'])
locs

In [ ]:
# df의 정수형 또는 실수형 변수만 선택하여 상관분석을 실행하고, 
# 유의확률이 0.05보다 작은지 여부를 데이터프레임으로 반환합니다.
df.loc[:, locs].apply(func = corr).lt(0.05)

### t-검정: MetColor

In [ ]:
# MetColor 범주별 Price의 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'Price', group = 'MetColor', method = 'shapiro')

In [ ]:
# (정규성 가정 만족) MetColor 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'Price', group = 'MetColor', method = 'levene')

In [ ]:
# MetColor 범주별 Price로 시리즈를 생성합니다.
sp1 = df['Price'][df['MetColor'].eq('0')]
sp2 = df['Price'][df['MetColor'].eq('1')]

In [ ]:
# 두 시리즈의 평균을 확인합니다.
print(sp1.mean())
print(sp2.mean())

In [ ]:
# 등분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = False)

In [ ]:
# 이분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = True)

In [ ]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = sp1, y = sp2)

### t-검정: Automatic

In [ ]:
# Automatic 범주별 Price의 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'Price', group = 'Automatic', method = 'shapiro')

In [ ]:
# (정규성 가정 만족) Automatic 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'Price', group = 'Automatic', method = 'levene')

In [ ]:
# Automatic 범주별 Price로 시리즈를 생성합니다.
sp1 = df['Price'][df['Automatic'].eq('0')]
sp2 = df['Price'][df['Automatic'].eq('1')]

In [ ]:
# 두 시리즈의 평균을 확인합니다.
print(sp1.mean())
print(sp2.mean())

In [ ]:
# 등분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = False)

In [ ]:
# 이분산 가정된 t-검정을 실행합니다.
pg.ttest(x = sp1, y = sp2, correction = True)

In [ ]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = sp1, y = sp2)

### 모평균 검정

In [ ]:
# 단일표본, 대응표본 t-검정에 사용할 데이터프레임 df1을 생성합니다.
df1 = pd.read_csv(filepath_or_buffer = 'https://bit.ly/ttest_dataset')

In [ ]:
# df1의 정보를 확인합니다.
df1.info()

In [ ]:
# df1의 처음 5행을 출력합니다.
df1.head()

In [ ]:
# df1 수치형 변수의 기술통계량을 확인합니다.
df1.describe()

In [ ]:
# before의 정규성 검정을 실행합니다.
stats.shapiro(x = df1['before'])[1]

In [ ]:
# 단일표본 t-검정을 실행합니다.(양측검정)
pg.ttest(x = df1['before'], y = 90)

In [ ]:
# 단일표본 t-검정을 실행합니다.(단측검정)
pg.ttest(x = df1['before'], y = 90, alternative = 'less')

In [ ]:
# df1의 열별 정규성 검정을 실행합니다.
df1.apply(func = lambda x: stats.shapiro(x)[1])

In [ ]:
# 대응표본 t-검정을 실행합니다.(양측검정)
pg.ttest(x = df1['before'], y = df1['after'], paired = True)

In [ ]:
# 대응표본 t-검정을 실행합니다.(단측검정)
pg.ttest(x = df1['before'], y = df1['after'], paired = True, alternative = 'greater')

### 분산분석: FuelType

In [ ]:
# FuelType 범주별 Price의 정규성 검정을 실행합니다.
pg.normality(data = df, dv = 'Price', group = 'FuelType', method = 'shapiro')

In [ ]:
# (정규성 가정 만족) FuelType 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, dv = 'Price', group = 'FuelType', method = 'levene')

In [ ]:
# 등분산 가정된 분산분석을 실행합니다.
pg.anova(data = df, dv = 'Price', between = 'FuelType')

In [ ]:
# 이분산 가정된 분산분석을 실행합니다.
pg.welch_anova(data = df, dv = 'Price', between = 'FuelType')

In [ ]:
# (정규성 가정 불만족) 크루스칼-왈리스 순위합 검정을 실행합니다.
pg.kruskal(data = df, dv = 'Price', between = 'FuelType')

### 사후분석(다중비교)

In [ ]:
# Bonferroni 방법으로 사후분석을 실행합니다.
sp.posthoc_ttest(a = df, val_col = 'Price', group_col = 'FuelType', 
                 p_adjust = 'bonferroni')

In [ ]:
# Tukey 방법으로 사후분석을 실행합니다.
sp.posthoc_tukey(a = df, val_col = 'Price', group_col = 'FuelType')

In [ ]:
# Scheffe 방법으로 사후분석을 실행합니다.
sp.posthoc_scheffe(a = df, val_col = 'Price', group_col = 'FuelType')

In [ ]:
# Tamhane 방법으로 사후분석을 실행합니다.
sp.posthoc_tamhane(a = df, val_col = 'Price', group_col = 'FuelType')

In [ ]:
# Nemenyi 방법으로 사후분석을 실행합니다.
sp.posthoc_nemenyi(a = df, val_col = 'Price', group_col = 'FuelType')

### 교차분석

In [ ]:
# 두 범주형 변수의 빈도수를 출력합니다.
pd.crosstab(index = df['MetColor'], columns = df['Automatic'])

In [ ]:
# 두 범주형 변수의 상대도수를 출력합니다.
pd.crosstab(index = df['MetColor'], columns = df['Automatic'], 
            normalize = 'index', margins = True)

In [ ]:
# 두 범주형 변수로 교차분석(카이제곱 검정)을 실행합니다.
test = pg.chi2_independence(data = df, x = 'MetColor', y = 'Automatic', 
                            correction = True)

In [ ]:
# 교차분석 결과를 확인합니다.
test

In [ ]:
# 피셔의 정확검정을 실행하면 오즈비와 유의확률을 반환합니다.
# test의 두 번째 원소인 관측값 행렬을 지정합니다.
stats.fisher_exact(table = test[1])

### 모비율 검정

In [ ]:
# 단일표본 모비율 검정을 실행합니다.
proportions_ztest(count = 30, nobs = 100, value = 0.2)

In [ ]:
# 두 표본의 모비율 차이 검정을 실행합니다.
proportions_ztest(count = [100, 150], nobs = [300, 500], value = 0.0)

### 변수 제거 및 외부 파일로 저장

In [ ]:
# 가설검정 결과를 반영하여 CC와 Automatic을 삭제합니다.
df = df.drop(columns = ['CC', 'Automatic'])

In [ ]:
# FuelType이 'CNG'인 건수가 매우 적고 Petrol, Diesel과의 
# 목표변수 평균에서 유의한 차이가 없으므로 삭제합니다.
df = df[df['FuelType'].ne('CNG')]

In [ ]:
# df의 행이름을 초기화합니다.
df = df.reset_index(drop = True)

In [ ]:
# df를 z 파일로 저장합니다.
joblib.dump(value = df, filename = 'Used_Cars_Price_Prep.z')

## End of Document